# Fit Evaluation

The `evaluate_fit` command calculates 6 error statistics, including the stats reported in the manuscript.

Smaller values are better.

In [ ]:
!sfacts evaluate_fit --simulation --outpath sim.eval_all_fits.tsv sim.world.nc sim.world.nc sim.filt.ss-0.fit.world.nc sim.filt.ss-0.fit2.world.nc sim.filt.fit3.world.nc
!column -t sim.eval_all_fits.tsv
# TODO: Discordance between pairwise metagenotype dissimilarity and predicted metagenotype dissimilarity.

We can visualize these on a per-sample or per-strain basis as well.

In [ ]:
import sfacts as sf
import xarray as xr

sim = sf.World.load('sim.world.nc')
fit = sf.World.load('sim.filt.ss-0.fit.world.nc')
fit = sf.World(fit.data.assign_coords(position=fit.position.astype(int)))

sim = sim.sel(position=fit.position.astype(int), sample=fit.sample.astype(int))

In [ ]:
mgen_error = sf.evaluation.metagenotype_error2(fit, discretized=True)
fwd_genotype_error = sf.evaluation.discretized_weighted_genotype_error(sim, fit)
rev_genotype_error = sf.evaluation.discretized_weighted_genotype_error(fit, sim)
bc_error = sf.evaluation.braycurtis_error(sim, fit)
unifrac_error = sf.evaluation.unifrac_error(sim, fit)
entropy_error = sf.evaluation.community_entropy_error(sim, fit)

In [ ]:
sf.plot.plot_community(
    sim,
    col_linkage_func=lambda w: w.metagenotype.linkage('sample'),
    row_linkage_func=lambda w: w.genotype.linkage('strain'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1].abs(),
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        fwd=fwd_genotype_error[1],
    )),
)
sf.plot.plot_community(
    fit,
    col_linkage_func=lambda w: w.metagenotype.linkage('sample'),
    row_linkage_func=lambda w: w.genotype.linkage('strain'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1].abs(),
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        rev=rev_genotype_error[1],
    )),
)
sf.plot.plot_metagenotype(
    fit,
    col_linkage_func=lambda w: w.metagenotype.linkage('sample'),
    row_linkage_func=lambda w: w.metagenotype.linkage('position'),
    col_colors_func=lambda w: xr.Dataset(dict(
        mg=mgen_error[1],
        bc=bc_error[1],
        uf=unifrac_error[1],
        ent=entropy_error[1].abs(),
    )),
    row_colors_func=lambda w: xr.Dataset(dict(
        _=w.metagenotype.mean_depth("position"),
    )),
)
sf.plot.plot_genotype(
    fit,
    row_linkage_func=lambda w: w.genotype.linkage('strain'),
    col_linkage_func=lambda w: w.metagenotype.linkage('position'),
    row_colors_func=lambda w: xr.Dataset(dict(
        rev=rev_genotype_error[1],
    )),
)
sf.plot.plot_genotype(
    sim,
    row_linkage_func=lambda w: w.genotype.linkage('strain'),
    col_linkage_func=lambda w: w.metagenotype.linkage('position'),
    row_colors_func=lambda w: xr.Dataset(dict(
        fwd=fwd_genotype_error[1],
    )),
)

In [ ]:
import scipy as sp
import matplotlib.pyplot as plt

mg_dist = sim.metagenotype.pdist()
uf_dist = fit.unifrac_pdist()

plt.scatter(sp.spatial.distance.squareform(mg_dist), sp.spatial.distance.squareform(uf_dist))
sp.stats.spearmanr(sp.spatial.distance.squareform(mg_dist), sp.spatial.distance.squareform(uf_dist))

## Epilogue: Selecting hyperparameters

This simulate/fit/evaluate loop gives us an opportunity to select better parameters than the defaults.
Likewise, in real data we can harness our expectations about
strain diversity (across samples),
strain heterogeneity (within samples), and
genotype ambiguity (a.k.a. fuzzyness)
to pick better parameters for a real-world dataset.

The regularization provided by priors is tunable and can be used to get more reasonable estimates.
Generally speaking, more of one type of regularization
(smaller values of `gamma_hyper`, `pi_hyper`, or `rho_hyper`) will result in
less effective regularization of the other two.